In [1]:
import numpy as np
from tqdm import tqdm
from time import time
import json, pickle, os, string, tqdm, kenlm, json
from collections import defaultdict, Counter
from itertools import groupby
import Levenshtein as Lev
import math


In [2]:
def lse(*args): # logsumexp trick
    a_max = max(args)
    lsp = math.log(sum(math.exp(a-a_max) for a in args))
    return a_max + lsp

In [3]:
#s1 = True text
#s2 = predicted text

def wer_(s1, s2):
    """
    Computes the Word Error Rate, defined as the edit distance between the
    two provided sentences after tokenizing to words.
    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """

    # build mapping of words to integers
    b = set(s1.split() + s2.split())
    word2char = dict(zip(b, range(len(b))))

    # map the words to a char array (Levenshtein packages only accepts
    # strings)
    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]
    
    return Lev.distance(''.join(w1), ''.join(w2))

def cer_(s1, s2):
    """
    Computes the Character Error Rate, defined as the edit distance.

    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """
    s1, s2, = s1.replace(' ', ''), s2.replace(' ', '')

    return Lev.distance(s1, s2)




#When using the above implementation, use the code belove to calculate the wer in percentatge: 
#pred = list of ouput prediction of model (it is the text) # example [" MY NAME IS HEMANT", " I AM A GOD"]
# total_wer = 0
# for x in range(len(pred)):
#     transcript, reference = data_[x][1], pred[x]
#     wer_inst = wer(transcript, reference)
#     total_wer += float(wer_inst)
# print("WER is : ",total_wer/len(pred),"%")
# wer_(pred,true)/len(pred.split(' '))*100

In [4]:
def ctc_best_path(out,labels):
    "implements best path decoding as shown by Graves"
    out = [labels[i] for i in np.argmax(out, axis=1) if i!=labels[-1]]
    o = ""
    for i,j in groupby(out):
        o = o + i
    return o.replace("_","")

### WORD LM Implementation

In [5]:
lm_w = kenlm.LanguageModel('/home/hemant/E2E_NER-Through-Speech/')

In [35]:

def sort_beam(ptot,k):
    if len(ptot) < k:
        return [i for i in ptot.keys()]
    else:
        dict_ = sorted(dict((v,k) for k,v in ptot.items()).items(),reverse=True)[:k]
        return [i[1] for i in dict_]

#using WORD LM
def ctc_beam_search(out,labels, prune=0.0001, k=20, lm=None,alpha=0.3,beta=12):
    "implements CTC Prefix Search Decoding Algo13.043478260869565%'rithm as shown by Graves"
    '''
    out = ctc output
    labels = string of labels
    prune = prune the ctc output
    k=beam-width
    lm=word age model used
    alpha,beta = hyper-parameters
    '''

    bc_i = 0 # blank/special charatcter index 
    F = out.shape[1]
    out = np.vstack((np.zeros(F), out))
    steps = out.shape[0]
    
    pb, pnb = defaultdict(Counter), defaultdict(Counter)
    pb[0][''], pnb[0][''] = 1, 0
    prev_beams = ['']
    for t in range(1,steps):
        pruned_alphabet = [labels[i] for i in np.where(out[t] > prune)[0]]
        for b in prev_beams:
            for c_t in pruned_alphabet:
                index = labels.index(c_t)
                #Collapsing case (copy case as the last character in the beam)
                if c_t == "_": #Extending with a blank
                    pb[t][b] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])   
                else:
                    i_plus = b + c_t
                    if len(b) > 0 and c_t == b[-1]: #Extending with the same character as the last one
                        pnb[t][b] += out[t][index]*pnb[t-1][b]
                        pnb[t][i_plus] += out[t][index]*pb[t-1][b]
                    #expanding the beam (extend case as the last character is different)
                    elif c_t == " " and len(b.replace(' ', '')) > 0 : # LM constraints
                        prob = [i[0] for i in lm.full_scores(i_plus,eos=False,bos=False)][-1]
                        lm_p = (10**prob)**alpha
                        pnb[t][i_plus] += lm_p*out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                    else:
                        pnb[t][i_plus] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                        
                    if i_plus not in prev_beams:
                        pb[t][i_plus] += out[t][index] * (pb[t - 1][i_plus] + pnb[t - 1][i_plus])
                        pnb[t][i_plus] += out[t][index] * pnb[t - 1][i_plus]

        ptot = pb[t] + pnb[t]
        for i in ptot.keys():
            ptot[i] = ptot[i]*(len(i)+1)**beta
        prev_beams = sort_beam(ptot,k)
    return prev_beams[0]

### CHARACTER LM Implementation


In [9]:
lm_c = kenlm.LanguageModel('/home/hemant/E2E_NER-Through-Speech/LM/3_char_gram.arpa')

In [10]:
def sort_beam(ptot,k):
    if len(ptot) < k:
        return [i for i in ptot.keys()]
    else:
        dict_ = sorted(dict((v,k) for k,v in ptot.items()).items(),reverse=True)[:k]
        return [i[1] for i in dict_]
#using CHARACTER LM
def ctc_beam_search_clm(out,labels, prune=0.001, k=20, lm=None,alpha=0.3,beta=12):
    "implements CTC Prefix Search Decoding Algorithm as shown by Graves"

    bc_i = 0 # blank/special charatcter index 
    F = out.shape[1]
    out = np.vstack((np.zeros(F), out))
    steps = out.shape[0]
    
    pb, pnb = defaultdict(Counter), defaultdict(Counter)
    pb[0][''], pnb[0][''] = 1, 0
    prev_beams = ['']
    for t in range(1,steps):
        pruned_alphabet = [labels[i] for i in np.where(out[t] > prune)[0]]
        for b in prev_beams:
            for c_t in pruned_alphabet:
                index = labels.index(c_t)
                #Collapsing case (copy case as the last character in the beam)
                if c_t == "_": #Extending with a blank
                    pb[t][b] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])  
                else:  # LM constraints
                    i_plus = b + c_t
                     #Extending with the same character as the last one
                    if len(b) > 0 and c_t == b[-1]:
                        pnb[t][b] += out[t][index]*pnb[t-1][b]
                        pnb[t][i_plus] += out[t][index]*pb[t-1][b]
                    #expanding the beam (extend case as the last character is different)
                    elif len(b.replace(' ', '')) > 0 :
                        prob = [i[0] for i in lm.full_scores(i_plus,eos=False,bos=False)][-1]
                        lm_p = 1#(10**prob)**alpha
                        pnb[t][i_plus] += lm_p*out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                    else:
                        pnb[t][i_plus] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])
        ptot = pb[t] + pnb[t]
        for i in ptot.keys():
            ptot[i] = ptot[i]*(len(i)+1)**beta
        prev_beams = sort_beam(ptot,k)
    return prev_beams

# IMPLEMANTATION

In [14]:
import os
os.chdir("/home/hemant/E2E_NER-Through-Speech/S2T/")
from opts import add_decoder_args, add_inference_args
from utils import load_model
import os
from ctc_decoders import *
import argparse
import numpy as np
import torch
from tqdm import tqdm
from data.data_loader import SpectrogramParser

from opts import add_decoder_args, add_inference_args
from utils import load_model

In [15]:
labels = "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ "

In [58]:
# decoding = 'greedy'
decoding = 'beam_w'
# decoding = 'beam_c'
prune = 0.0001
beam_width = 100
alpha = 1.02
beta = 12
lm = lm_w

torch.set_grad_enabled(False)
device = torch.device("cuda")
model = load_model(device, "/home/hemant/sopi_deep/models/deep/finetuen_sopi.pth")
spect_parser = SpectrogramParser(model.audio_conf, normalize=True)

torch.cuda.set_device(int(0))
with open("/home/hemant/sopi_deep/data/deepspeech/test/ata/test.csv","r") as f:
    csv = f.readlines()

total_cer, total_wer, num_tokens, num_chars = 0, 0, 0, 0
lm = lm_w
output = []
for i in tqdm(csv[:5]):
    audio_path, reference_path = i.split(",")

    spect = spect_parser.parse_audio(audio_path).contiguous()
    spect = spect.view(1, 1, spect.size(0), spect.size(1))
    spect = spect.to(device)

    input_sizes = torch.IntTensor([spect.size(3)]).int()
    out, output_sizes = model(spect, input_sizes)
    out = out.cpu().detach().numpy()[0]

    if decoding == "greedy": transcript = ctc_best_path(out,labels)
    elif decoding == "beam_w": transcript = ctc_beam_search(out,labels,prune,beam_width,lm,alpha,beta)
    elif decoding == "beam_c": transcript = ctc_beam_search_clm(out,labels,prune,beam_width,lm,alpha=alpha,beta=beta)
        
    with open(reference_path.replace("\n",""),"r") as f:
        reference = f.readline()
    output.append([transcript,reference])
    wer_inst = wer_(transcript,reference)
    cer_inst = cer_(transcript, reference)
    total_wer += wer_inst
    total_cer += cer_inst
    num_tokens += len(reference.split(' '))
    num_chars += len(reference.replace(' ', ''))
        
wer = (float(total_wer) / num_tokens)*100
cer = (float(total_cer) / num_chars)*100
print('Test Summary \t'
    'Average WER {wer:.3f}\t'
    'Average CER {cer:.3f}\t'.format(wer=wer, cer=cer))

100%|██████████| 5/5 [00:13<00:00,  2.66s/it]

Test Summary 	Average WER 72.000	Average CER 28.117	


In [22]:
audio_path,reference_path

('/home/hemant/sopi_deep/data/deepspeech/test/wav/en-122331-20191109-vyy-1251-1364.wav',
 '/home/hemant/sopi_deep/data/deepspeech/test/txt/en-122331-20191109-vyy-1251-1364.txt\n')

In [90]:
m = audio_path + "," +reference_path
with open("/home/hemant/junk/out.csv","w") as f:
    f.write(m)

In [29]:
wer_(transcript,reference)/len(reference.split(' '))*100

100.0

In [30]:
gred_txt = ctc_best_path(out,labels)
print(gred_txt)
wer_(gred_txt,reference)/len(reference.split(' '))*100

OUR PRODATIVITY AND OUR COMMITMENT TO OUR CLIENTS XXXUM TO MAKE SURE THAT WE NEED WHAT BE XXXUM PROMISED FOR THE DEADLINE


19.047619047619047

In [55]:
beam_txt=ctc_beam_search(out,labels,0.0001,k=100,lm=lm_w,alpha=1.2,beta=12)
print(beam_txt)
wer_(beam_txt,reference)/len(reference.split(' '))*100

OUR PRODUCTIVITY AND OUR COMMITMENT TO OUR CLIENTS XXXUM TO MAKE SURE THAT WE NEED WHAT WE XXXUM PROMISED FOR THE DEADLINE


9.523809523809524

In [ ]:
beam_txt=ctc_beam_search(out,labels,0.001,k=100,lm=lm_w,alpha=0,beta=0)
print(beam_txt)
wer_(beam_txt,reference)/len(reference.split(' '))*100

In [95]:
beam_txt=ctc_beam_search_clm(out,labels,0,k=10, lm_c = lm_c, alpha=0.1,beta=6)
print(beam_txt)
wer_(beam_txt,reference)/len(reference.split(' '))*100

NameError: name 'lm_c' is not defined

In [ ]:
    output.append([transcript,reference])

In [2]:
import numpy as np
import math
import collections

NEG_INF = -float("inf")

def make_new_beam():
  fn = lambda : (NEG_INF, NEG_INF)
  return collections.defaultdict(fn)

def logsumexp(*args):
  """
  Stable log sum exp.
  """
  if all(a == NEG_INF for a in args):
      return NEG_INF
  a_max = max(args)
  lsp = math.log(sum(math.exp(a - a_max)
                      for a in args))
  return a_max + lsp


In [59]:
def sort_beam(ptot,k):
    if len(ptot) < k:
        return [i for i in ptot.keys()]
    else:
        dict_ = sorted(dict((v,k) for k,v in ptot.items()).items(),reverse=True)[:k]
        return [i[1] for i in dict_]

#using WORD LM
def ctc_beam_search(out,labels, prune=0.0001, k=20, lm=None,alpha=0.3,beta=12):
    "implements CTC Prefix Search Decoding Algo13.043478260869565%'rithm as shown by Graves"

    bc_i = 0 # blank/special charatcter index 
    F = out.shape[1]
    out = np.vstack((np.zeros(F), out))
    steps = out.shape[0]
    
    pb, pnb = defaultdict(Counter), defaultdict(Counter)
    pb[0][''], pnb[0][''] = 1, 0
    prev_beams = ['']
    for t in range(1,steps):
        pruned_alphabet = [labels[i] for i in np.where(out[t] > prune)[0]]
        for b in prev_beams:
            for c_t in pruned_alphabet:
                index = labels.index(c_t)
                #Collapsing case (copy case as the last character in the beam)
                if c_t == "_": #Extending with a blank
                    pb[t][b] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])   
                else:
                    i_plus = b + c_t
                    if len(b) > 0 and c_t == b[-1]: #Extending with the same character as the last one
                        pnb[t][b] += out[t][index]*pnb[t-1][b]
                        pnb[t][i_plus] += out[t][index]*pb[t-1][b]
                    #expanding the beam (extend case as the last character is different)
                    elif c_t == " " and len(b.replace(' ', '')) > 0 : # LM constraints
                        prob = [i[0] for i in lm.full_scores(i_plus,eos=False,bos=False)][-1]
                        lm_p = (10**prob)**alpha
                        pnb[t][i_plus] += lm_p*out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                    else:
                        pnb[t][i_plus] += out[t][index]*(pb[t-1][b] + pnb[t-1][b])
                        
                    if i_plus not in prev_beams:
                        pb[t][i_plus] += out[t][index] * (pb[t - 1][i_plus] + pnb[t - 1][i_plus])
                        pnb[t][i_plus] += out[t][index] * pnb[t - 1][i_plus]


        ptot = pb[t] + pnb[t]
        for i in ptot.keys():
            ptot[i] = ptot[i]*(len(i)+1)**beta
        prev_beams = sort_beam(ptot,k)
    return prev_beams[0]

In [89]:
beam_txt=ctc_beam_search(out,labels,0,k=1,lm=lm_w,alpha=0,beta=0)
print(beam_txt)
wer_(beam_txt,reference)/len(reference.split(' '))*100

I THINK IS IS GOINGNA BE VORY ALD IDEA IN THE BEAST IN INTELLETION ALDATI BUSINESSES IN THE YEARS TO COME I DONTWT


46.15384615384615

In [ ]:
reference

In [104]:

def decode(probs, beam_size=100, blank=0):

  T, S = probs.shape
  probs = np.log(probs)

  beam = [(tuple(), (0.0, NEG_INF))]

  for t in range(T): # Loop over time
    next_beam = make_new_beam()

    for s in range(S): # Loop over vocab
      p = probs[t, s]
      for prefix, (p_b, p_nb) in beam: # Loop over beam
        if s == blank:
            n_p_b, n_p_nb = next_beam[prefix]
            n_p_b = logsumexp(n_p_b, p_b + p, p_nb + p)
            next_beam[prefix] = (n_p_b, n_p_nb)
            continue
        end_t = prefix[-1] if prefix else None
        n_prefix = prefix + (s,)
        n_p_b, n_p_nb = next_beam[n_prefix]
        if s != end_t:
          n_p_nb = logsumexp(n_p_nb, p_b + p, p_nb + p)
        else:
          n_p_nb = logsumexp(n_p_nb, p_b + p)
          
        next_beam[n_prefix] = (n_p_b, n_p_nb)
        if s == end_t:
          n_p_b, n_p_nb = next_beam[prefix]
          n_p_nb = logsumexp(n_p_nb, p_nb + p)
          next_beam[prefix] = (n_p_b, n_p_nb)
    beam = sorted(next_beam.items(),
            key=lambda x : logsumexp(*x[1]),
            reverse=True)
    beam = beam[:beam_size]

  best = beam[0]
  return best[0], -logsumexp(*best[1])


In [105]:
np.random.seed(0)

time = 50
output_dim = 20

probs = out # np.random.rand(time, output_dim)
probs = probs / np.sum(probs, axis=1, keepdims=True)

label, score = decode(probs)

print("Score {:.3f}".format(score))

Score 17.042


In [106]:
labels

"_'ABCDEFGHIJKLMNOPQRSTUVWXYZ "

In [107]:
out_ = ''
for i in label:
    out_+=labels[i]

In [108]:
out_

'I THINK IS IS GOINGNA BE VORY ALD IDEA IN THE BEAST IN INTELLETION ALDATI BUSINESSES IN THE YEARS TO COME I DONTWT'

In [109]:
wer_(out_,reference)/len(reference.split(' '))*100

46.15384615384615

In [113]:
# decoding = 'greedy'
decoding = 'beam_w'
# decoding = 'beam_c'
prune = 0
beam_width = 100
alpha = 0
beta = 0
lm = lm_w

torch.set_grad_enabled(False)
device = torch.device("cuda")
model = load_model(device, "/home/hemant/sopi_deep/models/deep/finetuen_sopi.pth")
spect_parser = SpectrogramParser(model.audio_conf, normalize=True)

torch.cuda.set_device(int(0))
with open("/home/hemant/sopi_deep/data/deepspeech/test/ata/test.csv","r") as f:
    csv = f.readlines()

total_cer, total_wer, num_tokens, num_chars = 0, 0, 0, 0
lm = lm_w
output = []
for i in tqdm(csv):
    audio_path, reference_path = i.split(",")

    spect = spect_parser.parse_audio(audio_path).contiguous()
    spect = spect.view(1, 1, spect.size(0), spect.size(1))
    spect = spect.to(device)

    input_sizes = torch.IntTensor([spect.size(3)]).int()
    out, output_sizes = model(spect, input_sizes)
    out = out.cpu().detach().numpy()[0]

    label, score = decode(out)
    transcript_ = ''
    for i in label:
        transcript_+=labels[i]
    
    with open(reference_path.replace("\n",""),"r") as f:
        reference = f.readline()
        
    print(transcript_)
    wer_inst = wer_(transcript_,reference)
    cer_inst = cer_(transcript_, reference)
    total_wer += wer_inst
    total_cer += cer_inst
    num_tokens += len(reference.split(' '))
    num_chars += len(reference.replace(' ', ''))
        
wer = (float(total_wer) / num_tokens)*100
cer = (float(total_cer) / num_chars)*100
print('Test Summary \t'
    'Average WER {wer:.3f}\t'
    'Average CER {cer:.3f}\t'.format(wer=wer, cer=cer))



  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 1/50 [00:10<08:42, 10.66s/it]

OUR PRODATIVITY AND OUR COMMITMENT TO OUR CLIENTS XXXUM TO MAKE SURE THAT WE NEED WHAT BE XXXUM PROMISED FOR THE DEADLINE




  4%|▍         | 2/50 [00:18<07:57,  9.94s/it]

I MEALMY IS IMPORTANT XXXUM I I WOULD TELL M SURMURPHE THAT XXXUM THOUH WE WOULD PAY BU




  6%|▌         | 3/50 [00:30<08:04, 10.32s/it]

...

SHEAP BECAUSE SHOL LEAR ANOTHER LANGUAE AS SI A ENGLISH THATS WILL OPEN EVERY




 98%|█████████▊| 49/50 [08:14<00:10, 10.14s/it]

OR IN EVERY COMPANY OR COUNTRI WITH THE ENGLISH WITHEN INGLES LANGUIGE XXXUM




100%|██████████| 50/50 [08:25<00:00, 10.12s/it]

I THINK IS IS GOINGNA BE VORY ALD IDEA IN THE BEAST IN INTELLETION ALDATI BUSINESSES IN THE YEARS TO COME I DONTWT
Test Summary 	Average WER 47.585	Average CER 23.864	
